In [ ]:
################################################
# Reconstructing the images from the segmented patches to generate a mask for the slide indicating the 
# affected region. To recreate the masks corresponding to the original slide, we resized the masks with 
# a downsample of 1/8 to the original dimension due to computational processing reasons.  
################################################

In [ ]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!pip install openslide-python
!pip install tifffile

#Install libraries
!pip install matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide
import os
import os.path
from PIL import Image
import math
import cv2

from tqdm import tqdm

In [ ]:
################################################
# Running utilities/stain_norm functions
################################################

# create_dir, read_slide, grays_filter, img_to_rgb, numpy2RGBpil
# isCenterMaskPositive, isPatchNotFullPositive, 

%run utilities_framework.ipynb

In [ ]:
####################################################,
# Base directory,
####################################################,

slide_num = '007'

base_path = '../slides_cropped/' + slide_num

all_patches_dir = base_path + '/all_patches/all/' #patches with tissues
roi_dir = base_path + '/all_patches/roi/' #patches with tissues

In [ ]:
#create_directory(all_patches_dir)
create_directory(roi_dir)

In [ ]:
################################################
# Read slide - check slide
################################################

slide_path = '../slides_to_test/' + 'test_'+ slide_num + '.tif'

slide = open_slide(slide_path)

level_to_plot=7

slide_image = read_slide(slide, 
                          x=0, 
                          y=0, 
                          level=level_to_plot,
                          width=slide.level_dimensions[level_to_plot][0], 
                          height=slide.level_dimensions[level_to_plot][1])

plt.figure(figsize=(3,3))
plt.imshow(slide_image)

#plt.figure(figsize=(5,5)) # specifying the overall grid size
#plt.subplot(1,2,1) # Number of images in the grid is 1*2 (1 line x 2 column + position on the grid)
#plt.imshow(slide_image)
#plt.subplot(1,2,2)    
#plt.imshow(img_normed)
#plt.show()

In [ ]:
################################################
# Patch configs
################################################

level = 8
itemSizeInPixels = 224 #patch_size
widthInPixels = slide.level_dimensions[LEVEL][0]
heightInPixels = slide.level_dimensions[LEVEL][1]

size_x = int(math.ceil(widthInPixels / itemSizeInPixels))
size_y = int(math.ceil(heightInPixels / itemSizeInPixels))

print("widthInPixels =", widthInPixels)
print("heightInPixels =", heightInPixels)
print("size_x, size_y =", size_x, size_y)

In [ ]:
for y in tqdm(range(level)):
    for x in tqdm(range(level)):
        patch_name = str(x) + "_" + str(y) + ".jpg"
        image = read_img(patch_name, all_patches_dir)
        
        columnStartPixelPosition = x * itemSizeInPixels
        columnEndPixelPosition = columnStartPixelPosition + (itemSizeInPixels - 1)
        rowStartPixelPosition = y * itemSizeInPixels
        rowEndPixelPosition = rowStartPixelPosition + (itemSizeInPixels - 1)
    

In [ ]:
def read_img(IMAGE_NAME, DIR):
    read = lambda imname: np.asarray(Image.open(imname))
    PATH = os.path.join(DIR,IMAGE_NAME)
    img = read(PATH)
    return img


size_y = int(math.ceil(heightInPixels / itemSizeInPixels))
size_x = int(math.ceil(widthInPixels / itemSizeInPixels))


partially_arr = np.zeros((level * itemSizeInPixels, level * itemSizeInPixels, 3), dtype=np.uint8)

for y in tqdm(range(size_y)): # round range up or down depending on where you want the black bar
    # row = np.array([])
    for x in range(size_x):  # round range up or down depending on where you want the black bar
        
        patch_name = str(x) + "_" + str(y) + ".jpg"
        image = read_img(patch_name, all_patches_dir)

        columnStartPixelPosition = x * itemSizeInPixels
        columnEndPixelPosition = columnStartPixelPosition + (itemSizeInPixels)
        rowStartPixelPosition = y * itemSizeInPixels
        rowEndPixelPosition = rowStartPixelPosition + (itemSizeInPixels)
        
        try:
            partially_arr[rowStartPixelPosition:rowEndPixelPosition,columnStartPixelPosition:columnEndPixelPosition] = image
        except:
            pass
        
resized_img = cv2.resize(partially_arr, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
patch_name = str(y) + '_' + str(x) + '.jpg'
plt.imsave(roi_dir + patch_name, resized_img)

 

plt.figure(figsize=(10,10)) # specifying the overall grid size
plt.imshow(partially_arr)

In [ ]:
def read_img(IMAGE_NAME, DIR):
    read = lambda imname: np.asarray(Image.open(imname))
    PATH = os.path.join(DIR,IMAGE_NAME)
    img = read(PATH)
    return img


size_y = int(math.ceil(heightInPixels / itemSizeInPixels))
size_x = int(math.ceil(widthInPixels / itemSizeInPixels))

size_j = int(math.ceil(size_y / level))
size_i = int(math.ceil(size_x / level))

for j in tqdm(range(size_j)):
    for i in range(size_i):
        partially_arr = np.zeros((level * itemSizeInPixels, level * itemSizeInPixels, 3), dtype=np.uint8)
        for y in range(level): # round range up or down depending on where you want the black bar
            for x in range(level):  # round range up or down depending on where you want the black bar

                crop_name = str(x+i) + "_" + str(x+j) + ".jpg"
                image = read_img(crop_name, all_patches_dir)

                columnStartPixelPosition = x * itemSizeInPixels
                columnEndPixelPosition = columnStartPixelPosition + (itemSizeInPixels)
                rowStartPixelPosition = y * itemSizeInPixels
                rowEndPixelPosition = rowStartPixelPosition + (itemSizeInPixels)

                try:
                    partially_arr[rowStartPixelPosition:rowEndPixelPosition,columnStartPixelPosition:columnEndPixelPosition] = image
                except:
                    pass

        resized_img = cv2.resize(partially_arr, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        patch_name = str(j) + '_' + str(i) + '.jpg'
        plt.imsave(roi_dir + patch_name, resized_img)

 
plt.figure(figsize=(10,10)) # specifying the overall grid size
plt.imshow(partially_arr)

In [ ]:
level = 64 # level 3
patch_size = 224
pedaco_arr_size = patch_size*level # 1792
channel = 3

img_w_original = slide.level_dimensions[0][0]
img_h_original = slide.level_dimensions[0][1]

final_img_w = slide.level_dimensions[6][0] # level 3 = 1/8 original
final_img_h = slide.level_dimensions[6][1]

originalSlice = np.zeros((pedaco_arr_size, pedaco_arr_size, channel), dtype=np.uint8)
final_image = np.zeros((final_img_w, final_img_h , channel), dtype=np.uint8)

x = 0
y = 0

i = 0
j = 0

qty_patches = 0

size_x = int(final_img_w/patch_size)
size_y = int(final_img_h/patch_size)

print(size_x, size_y)

for x in tqdm(range(size_x)):
    
    x_startLine = x*patch_size 
    x_endLine = (x+1)*patch_size
        
    for y in range(size_y):
            
        y_startLine = y*patch_size 
        y_endLine = (y+1)*patch_size
        
        i = 0
        j = 0

        for i in range(level):

            startLine = i*patch_size 
            endLine = (i+1)*patch_size 

            for j in range(level):

                qty_patches +=1
                patch_name = str((j*patch_size)+(y*patch_size*level)) + '_'+ str((i*patch_size)+(x*patch_size*level)) +'.jpg'
                
                try:
                    startColumn = j*patch_size
                    endColumn = (j+1)*patch_size
                    originalSlice[startLine:endLine,startColumn:endColumn] = read_img(patch_name, all_patches_dir)
                except:
                    pass
                      


        plt.figure(figsize=(5,5))
        plt.imshow(originalSlice)

        resized_img = cv2.resize(originalSlice, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        final_image[x_startLine:x_endLine,y_startLine:y_endLine] = resized_img
      
plt.figure(figsize=(10,10))
plt.imshow(final_image)
print('qty_patches =', qty_patches)

In [ ]:
level = 8 # level 3
patch_size = 224
pedaco_arr_size = patch_size*level # 1792
channel = 3

originalSlice = np.zeros((73728, 32598, channel), dtype=np.uint8)

for x in tqdm(range(3)):
    
    for y in range(3):

        for i in range(level):

            startLine = i*patch_size 
            endLine = (i+1)*patch_size 

            for j in range(level):
                
                patch_name = str((j*patch_size)+(y*patch_size*level)) + '_'+ str((i*patch_size)+(x*patch_size*level)) +'.jpg'
                startColumn = j*patch_size
                endColumn = (j+1)*patch_size
                originalSlice[startLine:endLine,startColumn:endColumn] = read_img(patch_name, all_patches_dir)

        #resized_img = cv2.resize(originalSlice, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        patch_name = str(y) + '_' + str(x) + '.jpg'
        plt.imsave(roi_dir + patch_name, pedaco_original)

plt.figure(figsize=(10,10))
plt.imshow(resized_img)
print('qty_patches =', qty_patches)